In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import datetime
import psycopg2 as pg
import pandas.io.sql as pd_sql

%config InlineBackend.figure_format = 'svg'
%matplotlib inline
sns.set(color_codes=True)
plt.style.use('seaborn-colorblind')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 5)

In [5]:
connection_args = {
    'host': 'localhost',  
    'dbname': 'flights',    
    'port': 5432          
}

connection = pg.connect(**connection_args)

In [47]:
query = """
    SELECT
        f.*,
        a.airline AS airline_name,
        o.airport AS origin_airport_name,
        o.city AS origin_airport_city,
        o.state AS origin_airport_state,
        o.latitude AS origin_airport_latitude,
        o.longitude AS origin_airport_longitude,
        d.airport AS destination_airport_name,
        d.city AS destination_airport_city,
        d.state AS destination_airport_state,
        d.latitude AS destination_airport_latitude,
        d.longitude AS destination_airport_longitude
    FROM flights AS f
        LEFT JOIN airlines AS a
            ON f.airline = a.iata_code
        LEFT JOIN airports AS o
            ON f.origin_airport = o.iata_code
        LEFT JOIN airports AS d
            ON f.destination_airport = d.iata_code
        ORDER BY RANDOM()
        LIMIT 100000;
"""

In [68]:
query = 'SELECT * FROM sample LIMIT 5'

In [69]:
pd_sql.read_sql(query, connection)

,year,month,day,day_of_week,airline,flight_number,tail_number,origin_airport,destination_airport,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft,weather_delay,airline_name,origin_airport_name,origin_airport_city,origin_airport_state,origin_airport_latitude,origin_airport_longitude,destination_airport_name,destination_airport_city,destination_airport_state,destination_airport_latitude,destination_airport_longitude
0,2015,7,28,2,OO,4640,N806SK,SLC,DFW,0830,0824,-6,15,0839,156,146,125,989,1144,6,1206,1150,-16,0,0,None,None,None,None,None,None,Skywest Airlines Inc.,Salt Lake City International Airport,Salt Lake City,UT,40.78839,-111.97777,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,32.89595,-97.03720
1,2015,9,11,5,B6,159,N187JB,BOS,PHL,0630,0630,0,11,0641,88,71,56,280,0737,4,0758,0741,-17,0,0,None,None,None,None,None,None,JetBlue Airways,Gen. Edward Lawrence Logan International Airport,Boston,MA,42.36435,-71.00518,Philadelphia International Airport,Philadelphia,PA,39.87195,-75.24114
2,2015,5,29,5,DL,1853,N967AT,MSN,ATL,0600,0552,-8,18,0610,125,134,103,707,0853,13,0905,0906,1,0,0,None,None,None,None,None,None,Delta Air Lines Inc.,Dane County Regional Airport,Madison,WI,43.13986,-89.33751,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,33.64044,-84.42694
3,2015,8,6,4,WN,1549,N421LV,SNA,OAK,1730,1729,-1,13,1742,80,77,59,371,1841,5,1850,1846,-4,0,0,None,None,None,None,None,None,Southwest Airlines Co.,John Wayne Airport (Orange County Airport),Santa Ana,CA,33.67566,-117.86822,Oakland International Airport,Oakland,CA,37.72129,-122.22072
4,2015,5,12,2,EV,4338,N16178,EWR,CMH,1210,1206,-4,16,1222,114,99,76,463,1338,7,1404,1345,-19,0,0,None,None,None,None,None,None,Atlantic Southeast Airlines,Newark Liberty International Airport,Newark,NJ,40.69250,-74.16866,Port Columbus International Airport,Columbus,OH,39.99799,-82.89188


In [52]:
sample_df.month.value_counts()

7     9011
8     8800
6     8764
5     8602
3     8477
12    8361
10    8329
4     8279
1     8039
9     7989
11    7959
2     7390
Name: month, dtype: int64

In [71]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   year                           100000 non-null  int64  
 1   month                          100000 non-null  int64  
 2   day                            100000 non-null  int64  
 3   day_of_week                    100000 non-null  int64  
 4   airline                        100000 non-null  object 
 5   flight_number                  100000 non-null  int64  
 6   tail_number                    99757 non-null   object 
 7   origin_airport                 100000 non-null  object 
 8   destination_airport            100000 non-null  object 
 9   scheduled_departure            100000 non-null  object 
 10  departure_time                 98518 non-null   object 
 11  departure_delay                98518 non-null   object 
 12  taxi_out                       

In [53]:
with open('sample_df.pickle', 'wb') as to_write:
    pickle.dump(sample_df, to_write)

In [24]:
query = """
    SELECT 
        origin_airport_name, 
        origin_airport_latitude, 
        origin_airport_longitude, 
        AVG(CAST(arrival_delay AS INTEGER)) AS origin_airport_avg_arrival_delay
    FROM combined
    GROUP BY origin_airport_name, origin_airport_latitude, origin_airport_longitude
    ORDER BY origin_airport_avg_arrival_delay DESC;
"""

In [26]:
origin_airport_avg_arrival_delay_df = pd_sql.read_sql(query, connection)

In [28]:
origin_airport_avg_arrival_delay_df = origin_airport_avg_arrival_delay_df.sort_values('origin_airport_avg_arrival_delay', ascending=False)

In [33]:
# Segment to look at top 100 worst origin_airports in terms of delays
top_delays_df = origin_airport_avg_arrival_delay_df.iloc[0:50]

In [34]:
top_delays_df.to_csv('top_airport_delays.csv', index=False)

In [35]:
top_delays_df

,origin_airport_name,origin_airport_latitude,origin_airport_longitude,origin_airport_avg_arrival_delay
0,Wilmington Airport,39.67872,-75.60653,24.06316
1,Gustavus Airport,58.42438,-135.70738,21.76316
2,Pago Pago International Airport (Tafuna Airport),14.33102,-170.71053,21.50000
3,St. Cloud Regional Airport,45.54532,-94.05834,19.55844
4,Jack Brooks Regional Airport (Southeast Texas ...,29.95083,-94.02069,16.15470
5,Aspen-Pitkin County Airport,39.22316,-106.86885,15.49249
6,Southwest Oregon Regional Airport (North Bend ...,43.41714,-124.24603,14.33333
7,Martha's Vineyard Airport,41.39303,-70.61433,12.98049
8,Eagle County Regional Airport,39.64257,-106.91770,12.40150
9,Trenton Mercer Airport,40.27669,-74.81347,11.99026
